In [1]:
import numpy as np
import pandas as pd
import pickle

In [3]:
meta_file = pd.read_csv('mole.csv')
reaction_file = pd.read_csv('Data_combines_0115.csv')
reaction_file.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
meta_file.head()

,meta_id,Gibbs_0,Std_Gibbs_0,Vec_0,1,2,3,4,5,6,...,118,119,120,121,122,123,124,125,126,127
0,udcpp_c,962.60144,4.861612,0.048866,-0.061648,-0.102033,0.107937,-0.058664,-0.035158,-0.003718,...,0.010449,-0.009979,-0.008570,-0.010946,0.087620,-0.035895,-0.040845,0.097057,0.017045,-0.001179
1,mql7_c,830.07500,4.261601,0.051076,-0.062318,-0.107261,0.126384,-0.056554,-0.039482,-0.011925,...,-0.000653,-0.014980,-0.012623,-0.003129,0.085750,-0.043235,-0.053473,0.092823,0.022405,-0.003495
2,mqn7_c,815.33276,4.194856,0.056053,-0.053472,-0.103328,0.112728,-0.063512,-0.025484,-0.011272,...,0.014445,-0.015437,-0.020543,-0.007141,0.085553,-0.037491,-0.054513,0.082303,0.016463,0.002097
3,2dmmq7_c,796.42050,4.109231,0.053506,-0.059303,-0.096238,0.109949,-0.060114,-0.030840,-0.003065,...,0.007757,-0.011240,-0.012409,-0.004290,0.089467,-0.043758,-0.050120,0.092414,0.009180,-0.004494
4,dtpcudcpp_c,753.04114,3.912832,0.052856,-0.062050,-0.096725,0.119363,-0.058011,-0.024000,-0.013419,...,0.005734,-0.001240,-0.014836,-0.007344,0.090392,-0.038148,-0.046038,0.090053,0.023485,0.002240


In [7]:
reaction_file.head()

,BSU,K,Enzyme,EC Numbers,EC_Number_firstdigit,reac_id,reac_name,reac_meta_name,reac_meta_value,compound,smiles,Gibbs,kcat,km
0,BSU00090,K00088,IMP dehydrogenase,1.1.1.205,1.0,IMPD,IMP dehydrogenase,h2o_c,-1.0,C00001,[H]O[H],-36.144756,NaN,additional information
1,BSU00090,K00088,IMP dehydrogenase,1.1.1.205,1.0,IMPD,IMP dehydrogenase,h_c,1.0,C00080,[H+],0.429061,NaN,additional information
2,BSU00090,K00088,IMP dehydrogenase,1.1.1.205,1.0,IMPD,IMP dehydrogenase,imp_c,-1.0,C00130,O[C@@H]1[C@@H](COP([O-])([O-])=O)O[C@H]([C@@H]...,-244.304930,NaN,additional information
3,BSU00090,K00088,IMP dehydrogenase,1.1.1.205,1.0,IMPD,IMP dehydrogenase,nad_c,-1.0,C00003,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,-338.381960,NaN,additional information
4,BSU00090,K00088,IMP dehydrogenase,1.1.1.205,1.0,IMPD,IMP dehydrogenase,nadh_c,1.0,C00004,NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP([O-])(=O)O...,-325.206970,NaN,additional information


In [8]:
enzyme_dict = dict()
enzyme_target = list()

for i in range(reaction_file.shape[0]):
    if reaction_file.loc[i, 'BSU'] in enzyme_dict: continue
    enzyme_dict[reaction_file.loc[i, 'BSU']] = len(enzyme_dict)
    if reaction_file.loc[i, 'kcat'] is np.nan or reaction_file.loc[i, 'kcat'] == 'additional information': continue
    enzyme_target.append([len(enzyme_dict)-1, 0, float(reaction_file.loc[i, 'kcat'])])
# enzyme_dict = dict(zip(enzyme_dict.values(), enzyme_dict.keys()))
n_enzyme = len(enzyme_dict)
print("number of enzyme:", n_enzyme)
print("enzyme target:", np.array(enzyme_target).shape)


cofactor = {'adp_c','atp_c','amp_c','nadp_c','nadph_c','nad_c','nadh_c','h2o_c','h2o_p','h_c','h_e','h_p','coa_c','ACP_c','pi_c','pi_p','ppi_c','cmp_c','q8h2_c','q8_c','mqn8_c','mql8_c','gtp_c','gdp_c','ctp_c','pi_e','udp_c','fadh2_c','gmp_c','ump_c','fe2_c','co2_c','co2_e','co2_p','o2_c'}
meta_dict = dict()
meta_feature = list()

for i in range(meta_file.shape[0]):
    if meta_file.loc[i, 'meta_id'] in cofactor: continue
    meta_dict[meta_file.loc[i, 'meta_id']] = len(meta_dict) + n_enzyme
    meta_feature.append(meta_file.iloc[i, 1:])
# meta_dict = dict(zip(meta_dict.values(), meta_dict.keys()))
n_meta = len(meta_dict)
print("number of meta:", n_meta)
print("dimension of meta feature:", len(meta_feature[0]))

number of enzyme: 393
enzyme target: (44, 3)
number of meta: 836
dimension of meta feature: 130


# Link
(node_id_source, node_id_target, edge_type_id, edge_weight).

In [9]:
link = pd.DataFrame(columns=("node_id_source", "node_id_target", "edge_type_id", "edge_weight"))
counter = 0
for i in range(reaction_file.shape[0]):
    BSU = reaction_file.loc[i, "BSU"]
    meta_name = reaction_file.loc[i, "reac_meta_name"]
    if BSU in enzyme_dict.keys() and meta_name in meta_dict.keys():
        reac_value = reaction_file.loc[i, "reac_meta_value"]
        if reac_value < 0:
            link.loc[len(link.index)] = np.array([meta_dict[meta_name], enzyme_dict[BSU], 0, -reac_value])
        elif reac_value > 0:
            link.loc[len(link.index)] = np.array([enzyme_dict[BSU], meta_dict[meta_name], 1, reac_value])
        counter += 1
display(link)

,node_id_source,node_id_target,edge_type_id,edge_weight
0,1033.0,0.0,0.0,1.0
1,0.0,1085.0,1.0,1.0
2,1.0,995.0,1.0,1.0
3,575.0,1.0,0.0,1.0
4,465.0,1.0,0.0,1.0
...,...,...,...,...
1414,390.0,1102.0,1.0,1.0
1415,1033.0,390.0,0.0,1.0
1416,1192.0,392.0,0.0,1.0
1417,392.0,971.0,1.0,1.0


# Node
(node_id, node_name, node_type_id, node_feature)

In [10]:
# NEED REVISION HERE
feature_size = len(meta_feature[0])
print("dimension of meta feature:", len(meta_feature[0]))

with open('BSU00090.pkl', 'rb') as f:
    data = pickle.load(f)
feature = data['representations']['pair'].flatten()[:feature_size]
feature.shape

dimension of meta feature: 130


FileNotFoundError: [Errno 2] No such file or directory: 'BSU00090.pkl'

In [42]:
# node (node_id, node_type_id, node_feature_1, node_feature_2, ...)
node = np.zeros((n_enzyme+n_meta, 2+feature_size))
for i in range(n_enzyme+n_meta):
    node[i, 0] = i
    if i < n_enzyme:
        node[i, 1] = 0
        # # enzyme_name = enzyme_dict
        # with open(enzyme_name+'.pkl', 'rb') as f:
        # with open('BSU00090.pkl', 'rb') as f:
        #     data = pickle.load(f)
        # feature = data['representations']['pair'].flatten()[:feature_size]
        node[i, 2:] = feature
    else:
        node[i, 1] = 1
        node[i, 2:] = meta_feature[i-n_enzyme]

In [43]:
node = pd.DataFrame(node)#, columns=("node_id", "node_type_id", range(130)))
node.rename(columns={0:"node_id", 1:"node_type_id"}, inplace = True)
display(node)

,node_id,node_type_id,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,0.0,0.0,22.418034,520.254578,3.085531,1.764717,-158.992661,47.613270,-61.161175,-107.856583,...,75.749290,-390.187347,73.389351,-81.145149,12.091395,-47.831512,-8.112387,1.706037,13.409246,-2.214712
1,1.0,0.0,22.418034,520.254578,3.085531,1.764717,-158.992661,47.613270,-61.161175,-107.856583,...,75.749290,-390.187347,73.389351,-81.145149,12.091395,-47.831512,-8.112387,1.706037,13.409246,-2.214712
2,2.0,0.0,22.418034,520.254578,3.085531,1.764717,-158.992661,47.613270,-61.161175,-107.856583,...,75.749290,-390.187347,73.389351,-81.145149,12.091395,-47.831512,-8.112387,1.706037,13.409246,-2.214712
3,3.0,0.0,22.418034,520.254578,3.085531,1.764717,-158.992661,47.613270,-61.161175,-107.856583,...,75.749290,-390.187347,73.389351,-81.145149,12.091395,-47.831512,-8.112387,1.706037,13.409246,-2.214712
4,4.0,0.0,22.418034,520.254578,3.085531,1.764717,-158.992661,47.613270,-61.161175,-107.856583,...,75.749290,-390.187347,73.389351,-81.145149,12.091395,-47.831512,-8.112387,1.706037,13.409246,-2.214712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1224.0,1.0,-740.252560,-2.848024,0.053427,-0.059173,-0.118821,0.137471,-0.064232,-0.036105,...,0.009586,-0.004133,-0.010821,-0.002006,0.097361,-0.047162,-0.050254,0.110523,0.020152,-0.008643
1225,1225.0,1.0,-764.989140,-2.960018,0.068900,-0.076324,-0.140355,0.140420,-0.079119,-0.033511,...,0.001803,-0.004426,-0.019078,-0.002578,0.115145,-0.057969,-0.058324,0.110015,0.021341,-0.005475
1226,1226.0,1.0,-980.059750,-3.933746,0.065787,-0.079164,-0.132009,0.146228,-0.070269,-0.044509,...,0.012550,-0.012039,-0.016554,-0.003405,0.112280,-0.054914,-0.057265,0.117445,0.025616,-0.000544
1227,1227.0,1.0,-1241.768100,-5.118625,0.069099,-0.085568,-0.150554,0.160313,-0.079892,-0.036877,...,-0.001537,-0.004240,-0.021818,-0.014600,0.122153,-0.056436,-0.054837,0.129391,0.023540,-0.006806


# Label
(node_id, node_type_id, node_label)

In [55]:
label_all = pd.DataFrame(np.array(enzyme_target))
label_all.rename(columns={0:"node_id", 1:"node_type_id", 2:"node_label"}, inplace = True)
display(label_all)

,node_id,node_type_id,node_label
0,2.0,0.0,1.410000e+00
1,18.0,0.0,4.000000e-02
2,20.0,0.0,3.500000e-02
3,42.0,0.0,5.210000e+05
4,49.0,0.0,8.300000e-04
5,50.0,0.0,1.190000e+03
6,58.0,0.0,1.740000e+00
7,70.0,0.0,2.600000e-01
8,92.0,0.0,1.360000e+00
9,94.0,0.0,4.000000e-01


## Label Train & Vali

In [57]:
label = label_all.sample(frac=0.5, replace=True, random_state=1)
display(label)

,node_id,node_type_id,node_label
37,346.0,0.0,18.8000
43,391.0,0.0,0.0140
12,114.0,0.0,16.5000
8,92.0,0.0,1.3600
9,94.0,0.0,0.4000
11,112.0,0.0,83.0000
5,50.0,0.0,1190.0000
15,147.0,0.0,0.0700
0,2.0,0.0,1.4100
16,172.0,0.0,25.4000


In [59]:
label_test = label_all[~label_all.index.isin(label.index)]
display(label_test)

,node_id,node_type_id,node_label
2,20.0,0.0,3.500000e-02
3,42.0,0.0,5.210000e+05
4,49.0,0.0,8.300000e-04
10,111.0,0.0,2.320000e+00
13,123.0,0.0,4.000000e-02
14,145.0,0.0,5.200000e+01
17,183.0,0.0,3.000000e-02
19,190.0,0.0,6.260000e+00
21,209.0,0.0,1.046000e+01
22,216.0,0.0,4.850000e+01


# Package

In [61]:
with open('link.pkl', 'wb') as f:
    pickle.dump(link, f)
with open('node.pkl', 'wb') as f:
    pickle.dump(node, f)
with open('label.pkl', 'wb') as f:
    pickle.dump(label, f)
with open('label_test.pkl', 'wb') as f:
    pickle.dump(label_test, f)